# Database Programming in Python

In [1]:
# Notebook set up
import sys, os, pathlib
sys.path.append(os.environ['BMESAHMETDIR'])
import pandas as pd
import sqlite3
import gzip
import bmes
from urllib.request import urlretrieve

BASE_PATH = pathlib.Path('.').resolve()
DB_PATH = pathlib.Path('/mnt/z/db/database.sqlite')

### Biolerplate Code for Database Programming

```python
import sqlite3

# Connect to the database
conn = sqlite3.connect( DB_PATH.joinpath('database.sqlite') )

# Create a cursor
# This will be used to execute SQL commands and fetch the results
cur = conn.cursor()

# ...
# SQL queries run using cur.execute( ... )
# ...

# Commit changes 
conn.commit()

# Close the connection when done so the connection is "released"
conn.close()
```

### Production Code Suggestions

- You should enclose the connections and queries in a `try ... except ... finally` block,
  so that you can handle any errors they raise
- Use transactions to group related queries together
- In a concurrent environment (e.g., web service), you should use locks to protect critical 
  operations.

### Conntext Manager

Use a context manager to handle the connection and cursor:

```python
class SQLite():
    def __init__(self, file):
        assert file.exists(), f"File {file} does not exist"
        assert file.endswith('.sqlite') or file.endswith('.db'), \
            f"File {file} is not a SQLite database"
        
        self.file = file

    def __enter__(self):
        self.conn = sqlite3.connect(self.file)
        self.conn.row_factory = sqlite3.Row
        return self.conn.cursor()

    def __exit__(self, type, value, traceback):
        self.conn.commit()
        self.conn.close()

with SQLite(DB_PATH.joinpath('database.sqlite')) as cur:
    print(cur.execute('SELECT sqlite_version();').fetchall()[0][0])
```

### In a "production" setting:
* You should enclose your connection and queries in "try ... except ...", so you handle any errors they raise.
* You should use transactions to group related queries.
* In a concurrent environment (e.g., web service) you should use locks to protect critical operations.

## Create / Drop Tables

In [2]:
# Connect to database
conn = sqlite3.connect(DB_PATH);
cur = conn.cursor();

# To drop a table:
conn.execute("DROP TABLE IF EXISTS students")

conn.execute("""
    CREATE  TABLE  IF  NOT  EXISTS  students (
        id INTEGER PRIMARY KEY,
        name VARCHAR(30), 
        birth DATE, 
        gpa FLOAT, 
        grad INTEGER
    );
""");

# Commit changes
conn.commit(); 

# Drop table
# Won't drop it here, because I will use the students table in the rest 
# of this tutorial.
# conn.execute("DROP TABLE IF EXISTS students")

# You should normally close when you are done with the connection, but 
# I will not close the connection here, so I can keep using it in the 
# rest of this tutorial.

## Insert / Delete Records

In [3]:
# Insert multiple rows
conn.execute("""
    INSERT INTO students (name, birth, gpa, grad) VALUES
        ('Anderson', '1987-10-22', 3.9, 2009),
        ('Jones', '1990-4-16', 2.4, 2012),
        ('Hernandez', '1989-8-12', 3.1, 2011),
        ('Chen', '1990-2-4', 3.2, 2011);
""")

# You dont have to specify all the fields
conn.execute("""
    INSERT INTO students (name) VALUES ('Ahmet');
""")

# To delete all rows:
# conn.execute("DELETE FROM students")

conn.commit()

## Select Queries

3 main elements:
* What you want (SELECT ...)
* Where it is found (FROM ...)
* How you want it filtered (WHERE ...)

Here, you need to use `cursor.execute()` instead of `connection.execute()`, 
because you want to fetch results.  
When you don't need to fetch results (i.e., CREATE / DROP / INSERT / DELETE / UPDATE),
you can use `connection.execute()`.

In [4]:
# Show all rows
cur.execute("SELECT * FROM students;");
rows = cur.fetchall()
for row in rows: print(row)

# Convert to pandas dataframe
df = pd.DataFrame.from_records(
    cur.execute("SELECT * FROM students").fetchall()
)
df.columns = [x[0] for x in cur.description]
display(df)

(1, 'Anderson', '1987-10-22', 3.9, 2009)
(2, 'Jones', '1990-4-16', 2.4, 2012)
(3, 'Hernandez', '1989-8-12', 3.1, 2011)
(4, 'Chen', '1990-2-4', 3.2, 2011)
(5, 'Ahmet', None, None, None)


,id,name,birth,gpa,grad
0,1,Anderson,1987-10-22,3.9,2009.0
1,2,Jones,1990-4-16,2.4,2012.0
2,3,Hernandez,1989-8-12,3.1,2011.0
3,4,Chen,1990-2-4,3.2,2011.0
4,5,Ahmet,None,NaN,NaN


In [5]:
# YOu can use the rows collection as a "list" and index it.
print(rows[0])
print(rows[0][2])

(1, 'Anderson', '1987-10-22', 3.9, 2009)
1987-10-22


### Definie `myselect()` to Make Queries Easier

In [6]:
# We'll be doing a lot of selectiona and printing, so let's define a 
# function to do that.
def select(query: str):

    cur.execute(query);
    rows = cur.fetchall()
    if len(rows) == 0:
        print("No results returned for SQL query")
    else:
        df = pd.DataFrame.from_records(rows)
        df.columns = [x[0] for x in cur.description]
        display(df)

    return

In [7]:
# Show just a few columns from a table:
# Retrieve entire contents of a table
select(" SELECT name, gpa FROM students; ")

,name,gpa
0,Anderson,3.9
1,Jones,2.4
2,Hernandez,3.1
3,Chen,3.2
4,Ahmet,NaN


In [8]:
# Filtering: only get a subset of the rows:
select("SELECT name, gpa FROM students WHERE gpa > 3.0;")

,name,gpa
0,Anderson,3.9
1,Hernandez,3.1
2,Chen,3.2


In [9]:
# Sorting: order the rows by a column
select("""
SELECT gpa, name, grad FROM students WHERE gpa > 3.0 ORDER BY gpa DESC;
""")

,gpa,name,grad
0,3.9,Anderson,2009
1,3.2,Chen,2011
2,3.1,Hernandez,2011


In [10]:
# Sorting: order by multiple columns
select("""
SELECT * FROM students ORDER BY grad DESC, gpa ASC;
""")

,id,name,birth,gpa,grad
0,2,Jones,1990-4-16,2.4,2012.0
1,3,Hernandez,1989-8-12,3.1,2011.0
2,4,Chen,1990-2-4,3.2,2011.0
3,1,Anderson,1987-10-22,3.9,2009.0
4,5,Ahmet,None,NaN,NaN


In [11]:
# Limiting: only get a certain number of rows:
select("""
SELECT name, gpa FROM students LIMIT 0,2;
""")

,name,gpa
0,Anderson,3.9
1,Jones,2.4


## Update / Delete

In [12]:
print("Before the update:")
select("SELECT * FROM students WHERE id = 2;")

# Update a row
conn.execute("""
UPDATE students
    SET gpa = 2.6, grad = 2013
    WHERE id = 2;
""")
conn.commit()

print("After the update:")
select("SELECT * FROM students WHERE id = 2;")

Before the update:


,id,name,birth,gpa,grad
0,2,Jones,1990-4-16,2.4,2012


After the update:


,id,name,birth,gpa,grad
0,2,Jones,1990-4-16,2.6,2013


In [13]:
print("Before the delete:")
select("SELECT * FROM students WHERE id = 2;")

conn.execute("DELETE FROM students WHERE id = 2;")

print("After the delete:")
select("SELECT * FROM students WHERE id = 2;")

# Let's add Jones back so future queries can use it again
conn.execute("""
INSERT INTO students (id, name, birth, gpa, grad) VALUES
    (2, 'Jones', '1990-4-16', 2.4, 2012);
""")
conn.commit()

Before the delete:


,id,name,birth,gpa,grad
0,2,Jones,1990-4-16,2.6,2013


After the delete:
No results returned for SQL query


## Joins

* **Join**: a query that merges the contents of 2 or more tables, and retrieves 
  information from the merged results.
  
### Join example: **many-to-one relationship**

* Students have advisors; add a new table describing faculty.

In [14]:
conn.execute("DROP TABLE IF EXISTS advisors");

conn.execute("""
CREATE TABLE advisors (
    id INTEGER PRIMARY KEY,
    name VARCHAR(30),
    title VARCHAR(30)
);
""")

conn.execute("""
INSERT INTO advisors (name, title) VALUES
    ('Fujimura', 'assocprof'),
    ('Bolosky', 'prof');
""")

conn.commit()

select("SELECT * FROM advisors;")

,id,name,title
0,1,Fujimura,assocprof
1,2,Bolosky,prof


### Foreign Key

* A **foreign key** is a field (or collection of fields) in one table that 
  uniquely identifies a row of another table or the same table. In simpler 
  words, the foreign key is defined in a second table, but it refers to the 
  primary key in the first table.

- Add a new column `advisor_id` to the students table, which is a foreign key
  to the faculty table.

In [15]:
# Add a foreign key to the students table
conn.execute("""
ALTER TABLE students ADD COLUMN advisor_id INTEGER;
""");

# Update the students table to set the advisor_id
conn.execute("""
UPDATE students 
    SET advisor_id = 2 
    WHERE id = 1;
""");

conn.execute("""
UPDATE students 
    SET advisor_id = 1 
    WHERE id = 2 OR id = 3 or id = 4;
""");

conn.commit()

select("SELECT * FROM students;")

,id,name,birth,gpa,grad,advisor_id
0,1,Anderson,1987-10-22,3.9,2009.0,2.0
1,2,Jones,1990-4-16,2.4,2012.0,1.0
2,3,Hernandez,1989-8-12,3.1,2011.0,1.0
3,4,Chen,1990-2-4,3.2,2011.0,1.0
4,5,Ahmet,None,NaN,NaN,NaN


**Perform the join query**

- Get students who are advised by Fujimura then get their names and gpas

In [16]:
select("""
SELECT students.name, students.gpa FROM students, advisors
    WHERE advisors.name = 'Fujimura' AND
          students.advisor_id = advisors.id;
""")

,name,gpa
0,Jones,2.4
1,Hernandez,3.1
2,Chen,3.2


You can give short names to the tables to be used within the query

In [17]:
select("""
SELECT s.name, s.gpa FROM students s, advisors a
    WHERE a.name = 'Fujimura' AND
          s.advisor_id = a.id;
""")

,name,gpa
0,Jones,2.4
1,Hernandez,3.1
2,Chen,3.2


You can also change the names of the fields that come back as results

In [18]:
select("""
SELECT a.id AS advisor_id, a.name AS advisor_name,
       s.id AS student_id, s.name AS student_name
FROM students s, advisors a
WHERE
    a.name = 'Fujimura' AND
    a.id = s.advisor_id;
""")

,advisor_id,advisor_name,student_id,student_name
0,1,Fujimura,2,Jones
1,1,Fujimura,3,Hernandez
2,1,Fujimura,4,Chen


### Join Example: many-to-many

In [19]:
# Create the courses table

conn.execute("DROP TABLE IF EXISTS courses;");
conn.execute("""
CREATE TABLE IF NOT EXISTS courses (
    id INTEGER PRIMARY KEY,
    number VARCHAR(30),
    name VARCHAR(32),
    quarter VARCHAR(32)
);
""")

# Populate the courses table
conn.execute("""
INSERT INTO courses (number, name, quarter) VALUES
    ('CS142', 'Web Stuff', 'Winter 2009'),
    ('ART101', 'Finger painting', 'Fall 2008'),
    ('ART101', 'Finger painting', 'Winter 2009'),
    ('PE204', 'Mud wrestling', 'Winter 2009');
""")

conn.commit()

select("SELECT * FROM courses;")

,id,number,name,quarter
0,1,CS142,Web Stuff,Winter 2009
1,2,ART101,Finger painting,Fall 2008
2,3,ART101,Finger painting,Winter 2009
3,4,PE204,Mud wrestling,Winter 2009


In [20]:
# Create the courses_students table
conn.execute("DROP TABLE IF EXISTS courses_students;");
conn.execute("""
CREATE TABLE IF NOT EXISTS courses_students (
    course_id INTEGER,
    student_id INTEGER,
    PRIMARY KEY (course_id, student_id)
);
""")

# Populate the courses_students table
conn.execute("""
INSERT INTO courses_students VALUES
    (1, 1), (3, 1), (4, 1), (1, 2),
    (2, 2), (1, 3), (2, 4), (4, 4);
""")
conn.commit()

select("SELECT * FROM courses_students;")

,course_id,student_id
0,1,1
1,3,1
2,4,1
3,1,2
4,2,2
5,1,3
6,2,4
7,4,4


Find all students who took a particular course (ART101). Get the student's name
and the term.

In [21]:
select("""
SELECT 
    s.name, 
    c.quarter
FROM 
    courses AS c,
    courses_students AS cs,
    students AS s
WHERE
    c.number = 'ART101' AND
    c.id = cs.course_id AND
    cs.student_id = s.id;
""")

,name,quarter
0,Anderson,Winter 2009
1,Jones,Fall 2008
2,Chen,Fall 2008


## Useful SQL functions

In [22]:
# Query the number of elements in a table (or any SELECT query)
select("SELECT count(*) AS numberofrows FROM students;")

,numberofrows
0,5


In [23]:
# Get the number of elements in a table, and extract the value
cur.execute("SELECT count(*) FROM students;").fetchall()[0][0]

5

In [24]:
select("SELECT name FROM students;")

select("SELECT name, count(*) FROM courses GROUP BY name;")

,name
0,Anderson
1,Jones
2,Hernandez
3,Chen
4,Ahmet


,name,count(*)
0,Finger painting,2
1,Mud wrestling,1
2,Web Stuff,1


In [25]:
select("SELECT distinct(name) FROM courses;")

,name
0,Web Stuff
1,Finger painting
2,Mud wrestling


In [26]:
# Close the connection to the database
conn.close()

## More Examples

In [37]:
# Download the a gzfile from MiRDB and read it line by line
url = "http://mirdb.org/download/miRDB_v6.0_prediction_result.txt.gz"
gzfile = '/mnt/z/db/miRDB_v6.0_prediction_result.txt.gz'
if not os.path.exists(gzfile):
    urlretrieve(url, '/mnt/z/db/miRDB_v6.0_prediction_result.txt.gz');

with gzip.open(gzfile, 'rb') as file:
    for i, line in enumerate(file):
        parts = line.strip().split(b'\t')
        print(parts)
        if i > 10: break

[b'cfa-miR-1185', b'XM_537211', b'59.3438099752']
[b'cfa-miR-1185', b'XM_536047', b'54.527']
[b'cfa-miR-1185', b'XM_005617022', b'55.1716326075']
[b'cfa-miR-1185', b'XM_014117861', b'57.4409058608']
[b'cfa-miR-1185', b'XM_014107884', b'57.1519']
[b'cfa-miR-1185', b'XM_005626419', b'67.0536']
[b'cfa-miR-1185', b'XM_005618203', b'62.64']
[b'cfa-miR-1185', b'NM_001252367', b'58.9597687709186']
[b'cfa-miR-1185', b'XM_005621885', b'78.529415636']
[b'cfa-miR-1185', b'XM_005622017', b'58.8579982864']
[b'cfa-miR-1185', b'XM_014116884', b'73.0734']
[b'cfa-miR-1185', b'XM_005628297', b'62.3335']


In [48]:
# Create a database to store the data
DB_PATH = pathlib.Path('/mnt/z/db/mirdb_test.sqlite')

conn = sqlite3.connect( DB_PATH )
cur = conn.cursor()

# Create a table to store the data
cur.execute("""
DROP TABLE IF EXISTS mirdb_predicted;
""")

conn.execute("""
CREATE TABLE IF NOT EXISTS mirdb_predicted (
    id INTEGER PRIMARY KEY,
    miRNA VARCHAR(30) COLLATE NOCASE,
    target VARCHAR(30) COLLATE NOCASE,
    score FLOAT
)
""")

# Read the gzfile line by line and insert the data into the database
with gzip.open(gzfile, 'rb') as file:
    for i, line in enumerate(file):
        parts = line.strip().split(b'\t')

        cur.execute(f"""
            INSERT INTO mirdb_predicted (miRNA, target, score) VALUES
            ('%s', '%s', '%f');
        """ % (parts[0].decode('utf-8'), parts[1].decode('utf-8'), float(parts[2])))

        if i == 100: 
            conn.commit()    
            break

# Look at the data
select("SELECT * FROM mirdb_predicted ORDER BY score DESC LIMIT 10;")

# Close the connection when done so the connection is "released"
conn.close()

,id,miRNA,target,score
0,75,cfa-miR-1185,XM_014111090,99.789705
1,83,cfa-miR-1185,XM_014111088,99.789705
2,51,cfa-miR-1185,XM_014118084,96.147970
3,87,cfa-miR-1185,XM_005622060,95.684780
4,22,cfa-miR-1185,XM_005625976,95.245695
5,95,cfa-miR-1185,XM_005617350,94.655800
6,88,cfa-miR-1185,XM_005629955,94.571031
7,70,cfa-miR-1185,XM_014118154,93.669588
8,54,cfa-miR-1185,XM_847370,92.766990
9,89,cfa-miR-1185,XM_005621823,91.587840
